# Chapter 1 - Exploratory Data Analysis(탐색적분석)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

## Estimates of Location( 위치추정)

> 데이터가 주어졌을때 데이터를 살펴보는 가장 기초적인 단계는 각 피쳐의 대표값을 구하는 것 
이는 중심경향성을 나타내는 추정값 

In [19]:
# 미국 각 주의 인구와 살인비율 데이터 

state = pd.read_csv("data/state.csv") 
state.head(8)

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA
5,Colorado,5029196,2.8,CO
6,Connecticut,3574097,2.4,CT
7,Delaware,897934,5.8,DE


#### 평균 - mean

In [7]:
state["Population"].mean()

6162876.3

#### 절사평균 - trim_mean

> 값들을 크기 순으로 정렬한 후 , 양끝에서 일정 개수의 값들을 삭제한 뒤 남은 값들을 가지고 구한 평균   
> 극단값의 영향을 제거

> 절사평균을 구하기 위해서는 scipy.stats 필요  
>SciPy는 과학기술계산을 위한 Python 라이브러리이다. NumPy, Matplotlib, pandas, SymPy와 연계되어 있다

In [11]:
from scipy.stats import trim_mean

trim_mean(state["Population"],0.1) # 0.1 은 각 끝에서 10%를 제외한다는 의미 

4783697.125

#### 가중평균  -weighted.mean
>각 데이터 값 x1에 사용자가 지정한 가중치 w1를 곱한 값들의 총합을 다시 가충치의 총합으로 나눈 것 

In [12]:
np.average(state["Murder.Rate"], weights = state["Population"])

4.445833981123393

#### 중간값 - median
> 데이터를 일렬로 정렬했을때 한가운데 위치하는 값  
> 데이터에 민감한 평균보다 중간값이 위치추정에 더 유리

In [13]:
state["Population"].median()

4436369.5

#### 가중중간값 - wquantiles
> 단순히 가운데 위치한 값이 아닌 어떤 위치를 기준으로 상위 절반의 가중치의 합이 하위절반의 가중치의 합과 동일한 위치의 값

>가중중간값을 사용하니위해 import weighted 필요

In [17]:
pip install wquantiles

Note: you may need to restart the kernel to use updated packages.


In [18]:
import wquantiles
wquantiles.median(state["Murder.Rate"], weights = state["Population"])

4.4

#### 특이값 
* 어떤 데이터 집합에서 다른 값들과 매우 멀리 떨어져 있는 값

> 일반적으로 위치추정에서 평균을 사용 but  
> 특이값들의 영향을 받지 않는 중간값, 가중중간값이 로버스트한 위치추정 방법 

## Estimates of Variability(변이추정)

> 변이는 데이터 값이 얼마나 밀집해 있는 혹은 퍼져있는지를 나타내는 산포도를 나타낸다.

In [20]:
state.head(8)

,State,Population,Murder.Rate,Abbreviation
0,Alabama,4779736,5.7,AL
1,Alaska,710231,5.6,AK
2,Arizona,6392017,4.7,AZ
3,Arkansas,2915918,5.6,AR
4,California,37253956,4.4,CA
5,Colorado,5029196,2.8,CO
6,Connecticut,3574097,2.4,CT
7,Delaware,897934,5.8,DE


#### 표준편차- std
> 분산의 제곱근 

In [21]:
state["Population"].std()

6848235.347401142

#### IQR(사분위범위)  - quantile(0.75) - quantile(0.25)
>  25번째 백분위수와 75번째 백분위수의 차이 

In [22]:
state["Population"].quantile(0.75) - state["Population"].quantile(0.25)

4847308.0

#### 중위절대편차- mad
>분산 ,표준편차 , 평균절대편차 모두 특잇값과 극단값에 로버스트하지 않다.   
> 중위절대편차는 로버스트   
> 계산하기 위해 statsmodels 패키지의 strong.scale.mad 함수 이용

In [24]:
from statsmodels import robust
robust.scale.mad(state["Population"])

3849876.1459979336